In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

from pyspark.sql.functions import col, row_number 
from pyspark.sql.window import Window

In [2]:
spark = (
    SparkSession.builder.appName("Collaborative Filtering")  # type: ignore
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.3.4")
    .config("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.DefaultAWSCredentialsProviderChain",
    )
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/05/08 22:49:03 WARN Utils: Your hostname, Camilo resolves to a loopback address: 127.0.1.1; using 172.29.121.68 instead (on interface eth0)
24/05/08 22:49:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camilo/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camilo/.ivy2/cache
The jars for the packages stored in: /home/camilo/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9fbfa06f-4224-4a5d-a135-cf66faf5479d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 277ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	--------------------------------

In [3]:
spark

In [4]:
# Leer el archivo Parquet
data = spark.read.parquet("s3a://amazon-reviews-eafit/sample/")
data_grouped = data.groupBy("customer_id").count().filter(col("count")>=3)
data = data.join(data_grouped,'customer_id','inner').drop('count')

24/05/08 22:49:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
# Obtener `customer_id` únicos
distinct_customer_ids = data.select("customer_id").distinct()

# Tomar una muestra aleatoria de 500 `customer_id`
sampled_customer_ids = distinct_customer_ids.sample(False, 0.5)  # Ajusta la fracción según necesidades
sampled_customer_ids = sampled_customer_ids.limit(500)

# Convertir la columna de `customer_id` en una lista de Python
list_of_sampled_ids = [row['customer_id'] for row in sampled_customer_ids.collect()]

# Filtrar el DataFrame original para incluir solo las filas con los `customer_id` muestreados
data = data.filter(col("customer_id").isin(list_of_sampled_ids))

In [6]:
data.count()

1729

In [7]:
indexer = StringIndexer(inputCol="product_id", outputCol="item_id")

data = indexer.fit(data).transform(data)

In [8]:
windowSpec = Window.partitionBy("customer_id").orderBy(col("review_date").desc())
data = data.withColumn("index", row_number().over(windowSpec))

training = data.where(col("index") >= 3)
validation = data.where(col("index") == 2)
test = data.where(col("index") <= 1)

In [9]:
training.count()

729

In [10]:
validation.count()

500

In [11]:
test.count()

500

In [12]:
# Build the recommendation model using ALS on the training data
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol="customer_id",
    itemCol="item_id",
    ratingCol="star_rating",
    seed= 42,
    nonnegative = True,
    rank=1,
    coldStartStrategy='drop'
)
model = als.fit(training)

24/05/08 22:49:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/08 22:49:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [13]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

print(predictions.count())
evaluator_rmse = RegressionEvaluator(
    metricName="rmse", labelCol="star_rating", predictionCol="prediction"
)
rmse = evaluator_rmse.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

evaluator_mae = RegressionEvaluator(
    metricName="mae", labelCol="star_rating", predictionCol="prediction"
)
mae = evaluator_mae.evaluate(predictions)
print(f"Mean absolute error = {mae}")

# Generate top 10 product recommendations for each user
# userRecs = model.recommendForAllUsers(10)

# Show top 10 recommendations for each user
# print(userRecs.show())

summary = spark.createDataFrame(
    [
        Row(metric="RMSE", value=rmse),
        Row(metric="MAE", value=mae)
    ]
)

(
    summary.coalesce(1)  # Save as a single CSV file
    .write.mode("overwrite")
    .option("header", "true")
    .csv("../data/results/localALS.csv")
)

4


Root-mean-square error = 3.554783486042441


Mean absolute error = 2.6852672398090363


In [14]:
spark.stop()